# TODO:
- Check which crimes are correl. with socio-economic status (proxy: race (and sex and age?))
- Check whether race/age/sex is correl. with COMPAS scores
- Check whether race/age/sex is correl. with OUR scores
- Try out some other models as well (SVMs, Random Forests, Naive Bayes) 

# Trying stuff out
Import our tools

In [195]:
import pandas as pd

Load in the relevant data, parsing the date fields to a format pandas understands on the way.

In [196]:
df = pd.read_csv(
    'compas-scores.csv',
    usecols=[
        'sex',
        'age',
        #'age_cat',
        'race',
        'juv_fel_count',
        #'decile_score',
        'juv_misd_count',
        'juv_other_count', 
        'priors_count',
        'days_b_screening_arrest',
        'c_jail_in',
        'c_jail_out',
        'c_offense_date',
        'c_arrest_date',
        'c_days_from_compas', 
        'c_charge_degree',
        'c_charge_desc',
        'is_recid',
        #'num_r_cases',
        #'r_charge_degree', 
        #'r_days_from_arrest', 
        #'r_offense_date',
        #'r_charge_desc', 
        #'r_jail_in', 
        #'r_jail_out', 
        #'is_violent_recid',
        #'num_vr_cases', 
        #'vr_charge_degree', 
        #'vr_offense_date',
        #'vr_charge_desc', 
        #'v_decile_score',
        #'v_score_text', 
        #'v_screening_date', 
        #'score_text', 
    ],
    parse_dates=[
        'c_jail_in', 
        'c_jail_out', 
        'c_offense_date', 
        'c_arrest_date', 
        #'r_offense_date', 
        #'r_jail_in', 
        #'r_jail_out', 
        #'vr_offense_date',
        #'v_screening_date'
    ]
)

In [197]:
df = pd.read_csv(
    'compas-scores.csv',
    usecols=[
        'sex',
        'age',
        #'age_cat',
        'race',
        'juv_fel_count', 
        #'decile_score',
        'juv_misd_count',
        'juv_other_count', 
        'priors_count',
        'days_b_screening_arrest',
        'c_jail_in',
        'c_jail_out',
        'c_offense_date',
        'c_arrest_date', 
        'c_days_from_compas', 
        'c_charge_degree',
        'c_charge_desc', 
        'is_recid',
        #'num_r_cases',
        #'r_charge_degree', 
        #'r_days_from_arrest', 
        #'r_offense_date',
        #'r_charge_desc', 
        #'r_jail_in', 
        #'r_jail_out', 
        'is_violent_recid',
        #'num_vr_cases', 
        #'vr_charge_degree', 
        #'vr_offense_date',
        #'vr_charge_desc', 
        #'v_decile_score',
        #'v_score_text', 
        #'v_screening_date', 
        #'score_text', 
    ],
    parse_dates=[
        'c_jail_in', 
        'c_jail_out', 
        'c_offense_date', 
        'c_arrest_date', 
        #'r_offense_date', 
        #'r_jail_in', 
        #'r_jail_out', 
        #'vr_offense_date',
        #'v_screening_date'
    ]
)

In [198]:
print(df.shape)
df = df[df['c_days_from_compas'] <= 10]
print(df.shape)
print(df.columns)
df['c_charge_desc'] = df.c_charge_desc.apply(lambda x: str(x).lower())
print(df.c_charge_desc[['agg' in desc for desc in df.c_charge_desc.tolist()]].unique())

(11757, 17)
(8938, 17)
Index(['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count',
       'juv_other_count', 'priors_count', 'days_b_screening_arrest',
       'c_jail_in', 'c_jail_out', 'c_offense_date', 'c_arrest_date',
       'c_days_from_compas', 'c_charge_degree', 'c_charge_desc', 'is_recid',
       'is_violent_recid'],
      dtype='object')
['aggravated assault w/firearm' 'agg fleeing and eluding'
 'agg battery grt/bod/harm' 'aggravated battery / pregnant'
 'aggravated assault w/dead weap' 'aggrav battery w/deadly weapon'
 'aggrav stalking after injunctn'
 'aggravated battery (firearm/actual possession)'
 'agg fleeing/eluding high speed' 'aggravated battery'
 'aggrav child abuse-agg battery' 'aggravated assault'
 'agg assault w/int com fel dome' 'aggrav child abuse-causes harm'
 'stalking (aggravated)' 'aggravated assault w/o firearm'
 'aggr child abuse-torture,punish' 'aggravated battery on 65/older'
 'aggravated battery (firearm)' 'aggravated manslaughter child'
 'agg abuse 

In [199]:

np.savetxt('unique_c_charge_desc_lowercase.txt', df['c_charge_desc'].unique(), fmt='%s')

Prune non-violent out from the dataset

In [200]:
possibly_violent = ('assault')
violent_kw = {'aggravated', 'battery', 'burglary', 'agg', 'violence', 'manslaughter', 'murder', 'bodily',
                    'armed'}
non_violent_kw = {'w/o violence', 'without violence', 'no bodily harm'}

def violent_charge(x):
    s = set(x.split())
    if s - non_violent_kw != s: return False
    if s - violent_kw != s: return True
    else: return False
    
df = df[df.c_charge_desc.apply(violent_charge)]
print(df.c_charge_desc)

0           aggravated assault w/firearm
2         felony battery w/prior convict
6                                battery
7              possession burglary tools
9                                battery
14                               battery
15                               battery
16                               battery
20                               battery
25         battery on law enforc officer
29        lewd/lasc battery pers 12+/<16
32                               battery
40        felony battery w/prior convict
42                               battery
47       burglary structure assault/batt
49               agg fleeing and eluding
51                               battery
52                               battery
58        viol injunct domestic violence
59          burglary unoccupied dwelling
62            burglary dwelling occupied
64                               battery
65                               battery
78          attempt armed burglary dwell
83              

In [201]:
df['c_charge_desc'].value_counts().to_csv('unique_c_charge_desc_lowercase.csv')

Transform all string data to lowercase and remove extra whitespace. This handles some problematic cases where we have e.g. 'Id theft' and 'ID Theft' as separate labels.

In [202]:
df = df.applymap(lambda s : s.lower().strip() if type(s) == str else s)
df['c_charge_desc'].value_counts().to_csv('cadasdas.csv')

Transform all string columns to groups of binary columns. This is probably not the smartest way to go about this, given that we have so many crime labels. The smart thing to do would probably be some type of a crime classification system, e.g. is_violent, is_with_weapon, etc. But that would require a lot of manual work.

In [203]:
df = pd.get_dummies(df)

Check if we have any more object columns (should be none)

In [204]:
df.dtypes.value_counts()

uint8             76
int64              7
datetime64[ns]     4
float64            2
dtype: int64

Calculate jail stay lengths

In [205]:
df['c_days_in_jail'] = (df['c_jail_out'] - df['c_jail_in']).dt.days.fillna(0).astype(int)
#df['r_days_in_jail'] = (df['r_jail_out'] - df['r_jail_in']).dt.days.fillna(0).astype(int)

Drop date columns

In [206]:
df.drop([
        'c_jail_in', 
        'c_jail_out', 
        'c_offense_date', 
        'c_arrest_date', 
        #'r_offense_date', 
        #'r_jail_in', 
        #'r_jail_out', 
        #'vr_offense_date',
        #'v_screening_date'
    ],
    axis='columns',
    inplace=True
)

Check dtypes again, should be no dates remaining

In [207]:
df.dtypes.value_counts()

uint8      76
int64       8
float64     2
dtype: int64

Still a few floats, see what that is all about

In [208]:
df.select_dtypes(include=['float64'])

,days_b_screening_arrest,c_days_from_compas
0,-1.0,1.0
2,-1.0,1.0
6,0.0,0.0
7,-1.0,1.0
9,-1.0,1.0
14,0.0,0.0
15,0.0,0.0
16,0.0,0.0
20,0.0,0.0
25,-1.0,1.0


At least two look like only NaNs, check to be sure

In [209]:
#print(df['num_vr_cases'].value_counts())
#print(df['num_r_cases'].value_counts())

Yeah, NaNs only. We can drop those. The others look pretty meaningless as well, so drop those too.

In [210]:
df.drop([
        #'num_r_cases',
        #'num_vr_cases',
        'days_b_screening_arrest',
        'c_days_from_compas',
        #'r_days_from_arrest'

    ],
    axis='columns',
    inplace=True
)

Check if we have any more NaNs hanging around

In [211]:
for col in df:
    count = len(df[col]) - df[col].count()
    if count:
        print(col, count)

Should be no more NaNs to go, so we can continue to learning.

Split to target and explanatory variables

In [222]:
print(df.columns)
predicted_variable = 'is_violent_recid'
df_ = df.query(predicted_variable + ' in (0, 1)')
print('negative samples: ', len(df) - len(df_))
X = df_.loc[:, df_.columns != predicted_variable]
y = df_.loc[:, df_.columns == predicted_variable]
print(X.shape, y.shape)
n_pos = y[predicted_variable].sum()
print(n_pos / (y.shape[0]))

Index(['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count',
       'priors_count', 'is_recid', 'is_violent_recid', 'sex_female',
       'sex_male', 'race_african-american', 'race_asian', 'race_caucasian',
       'race_hispanic', 'race_native american', 'race_other',
       'c_charge_degree_f', 'c_charge_degree_m',
       'c_charge_desc_agg abuse elderlly/disabled adult',
       'c_charge_desc_agg assault w/int com fel dome',
       'c_charge_desc_agg battery cause bodily',
       'c_charge_desc_agg battery grt/bod/harm',
       'c_charge_desc_agg fleeing and eluding',
       'c_charge_desc_agg fleeing/eluding high speed',
       'c_charge_desc_aggrav battery w/deadly weapon',
       'c_charge_desc_aggrav child abuse-agg battery',
       'c_charge_desc_aggravated animal cruelty',
       'c_charge_desc_aggravated assault',
       'c_charge_desc_aggravated assault w/dead weap',
       'c_charge_desc_aggravated assault w/firearm',
       'c_charge_desc_aggravated assault w/o firear

Split to train and test sets

In [223]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1, random_state=12)

Train a ridge regression model and check the $R^2$ score. Score of 1 is best possible, values can range to $-\inf$. RidgeCV automatically cross validates (leave-one-out) to figure out the best hyperparameter $\alpha$ 

In [240]:
from sklearn import linear_model, ensemble, neural_network
from sklearn.metrics import precision_recall_fscore_support as pr_f1, accuracy_score as acc, matthews_corrcoef
import numpy as np


def print_score(modelname, y, y_hat):
    accuracy = acc(y, y_hat)
    matt = matthews_corrcoef(y.values.ravel(), y_hat)
    p, r, f1, sup = pr_f1(y, y_hat)
    print(modelname)
    print('accuracy:', acc(y, y_hat))
    print('precision:', tuple(p))
    print('recall:', tuple(r))
    print('f1:', tuple(f1))
    print('Matthew:', matt)
    print('')
    
naive_y = np.zeros(len(y_test))
print_score('Naive', y_test, naive_y)

model = linear_model.LogisticRegression()
model.fit(X_train, y_train.values.ravel())
print_score('Logreg', y_test, model.predict(X_test))

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train, y_train.values.ravel())
print_score('KNN', y_test, model.predict(X_test))

model = neural_network.MLPClassifier(hidden_layer_sizes=((100,)*20), solver='adam', activation='logistic')
model.fit(X_train, y_train.values.ravel())
print_score('MLP', y_test, model.predict(X_test))


/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Naive
accuracy: 0.9090909090909091
precision: (0.9090909090909091, 0.0)
recall: (1.0, 0.0)
f1: (0.9523809523809523, 0.0)
Matthew: 0.0

Logreg
accuracy: 0.9058441558441559
precision: (0.9087947882736156, 0.0)
recall: (0.9964285714285714, 0.0)
f1: (0.9505962521294719, 0.0)
Matthew: -0.01804807189211084

KNN
accuracy: 0.9058441558441559
precision: (0.9087947882736156, 0.0)
recall: (0.9964285714285714, 0.0)
f1: (0.9505962521294719, 0.0)
Matthew: -0.01804807189211084

MLP
accuracy: 0.9090909090909091
precision: (0.9090909090909091, 0.0)
recall: (1.0, 0.0)
f1: (0.9523809523809523, 0.0)
Matthew: 0.0



In [241]:
c = model.predict(X_test)
res = model.predict_proba(X_test)
[(r[1], c) for r, c in zip(res, c)]

[(0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 (0.09921774157649195, 0),
 

In [216]:
!jt -r 

Reset css and font defaults in:
/u/48/sarsas2/unix/.jupyter/custom &
/u/48/sarsas2/unix/.local/share/jupyter/nbextensions


That's... not horrible. Let's check which features have the greatest effect on the thing. 

In [217]:
#coeffs = [(coef, col) for col, coef in zip(X.columns, model.coef_[0])]

import numpy as np
coefs = [np.mean(c) for c in model.coefs_[0]]
coeffs = [(coef, col) for col, coef in zip(X.columns, coefs)] # for neural net
coeffs = sorted(coeffs, reverse=True)
for a, b in coeffs: print(a, b)


0.027421200300390372 c_charge_desc_burglary unoccupied dwelling
0.024302452952596552 race_other
0.022854312568178506 age
0.02221384203428667 c_charge_degree_f
0.021704296243018707 sex_female
0.018958302661877213 c_charge_desc_aggravated assault w/dead weap
0.016710824912836566 c_charge_desc_aggravated battery on 65/older
0.015455299966418652 c_charge_desc_aggravated battery / pregnant
0.015246295823307578 race_hispanic
0.013474260522652805 c_charge_desc_attempted aggravated battery
0.013425663434782716 c_charge_desc_burglary conveyance unoccup
0.013208881093127631 c_charge_desc_neglect child / no bodily harm
0.012988547770352292 c_charge_desc_sex battery deft 18+/vict 11-
0.012382049131515718 c_charge_desc_burglary assault/battery armed
0.011817963141059746 race_caucasian
0.01063488503716373 c_charge_desc_battery emergency care provide
0.010499285131196093 c_charge_desc_battery on a person over 65
0.010428266231731336 c_charge_desc_agg fleeing and eluding
0.01026347925129387 c_charge_d

In [218]:
#coeffs = [(coef, col) for col, coef in zip(X.columns, model.coef_[0])]
#coeffs = sorted(coeffs, reverse=True)

for a, b in coeffs[:20]: print(a, b)
    
print('...')

orig = {col:coef for coef, col in coeffs}
abss = sorted([(abs(coef), col) for coef, col in coeffs])
near_zero = sorted([(orig[col], col) for coef, col in abss[:50]], reverse=True)
for a, b in near_zero: print(a, b)


print('...')

for a, b in coeffs[-20:]: print(a, b)

0.027421200300390372 c_charge_desc_burglary unoccupied dwelling
0.024302452952596552 race_other
0.022854312568178506 age
0.02221384203428667 c_charge_degree_f
0.021704296243018707 sex_female
0.018958302661877213 c_charge_desc_aggravated assault w/dead weap
0.016710824912836566 c_charge_desc_aggravated battery on 65/older
0.015455299966418652 c_charge_desc_aggravated battery / pregnant
0.015246295823307578 race_hispanic
0.013474260522652805 c_charge_desc_attempted aggravated battery
0.013425663434782716 c_charge_desc_burglary conveyance unoccup
0.013208881093127631 c_charge_desc_neglect child / no bodily harm
0.012988547770352292 c_charge_desc_sex battery deft 18+/vict 11-
0.012382049131515718 c_charge_desc_burglary assault/battery armed
0.011817963141059746 race_caucasian
0.01063488503716373 c_charge_desc_battery emergency care provide
0.010499285131196093 c_charge_desc_battery on a person over 65
0.010428266231731336 c_charge_desc_agg fleeing and eluding
0.01026347925129387 c_charge_d

Let's check the non-crime label ones specifically

In [219]:
for a, b in coeffs: print(a, b) if 'charge_desc' not in b else ...

0.024302452952596552 race_other
0.022854312568178506 age
0.02221384203428667 c_charge_degree_f
0.021704296243018707 sex_female
0.015246295823307578 race_hispanic
0.011817963141059746 race_caucasian
0.006213834475870968 is_recid
0.0031217694714528997 c_days_in_jail
0.001050918048959672 race_asian
0.00013774694228521508 race_native american
-0.0007053553358797226 juv_other_count
-0.00255088791095787 sex_male
-0.002683240358355633 race_african-american
-0.008805753901131088 priors_count
-0.01252300188636484 c_charge_degree_m
-0.013041410006718277 juv_misd_count
-0.020200651222982126 juv_fel_count


Try linear support vector regression next

In [220]:
from sklearn import svm

#model = svm.LinearSVR(epsilon=0.5)
#scores = model_selection.cross_val_score(model, X, y.values.ravel(), cv=10, n_jobs=2)
#scores

Marginally better, still pretty bad. Let's try support vector regression with an RBF. In principle it has an infinite-dimensional faeture space so that should get to something reasonable at the cost of possibly overfitting.

In [221]:
from sklearn import svm

#model = svm.SVR(epsilon=0.5)
#scores = model_selection.cross_val_score(model, X, y.values.ravel(), cv=10, n_jobs=3)
#scores

It's... not bad(?) considering we are using $\frac{1}{3}$ of the data for testing on each round of the cross validation. The resulting $R^2$ values indicate $R \approx 0.6$ which is by no means an abysmal degree of correlation. At the same time, we would expect a significantly higher degree of success based on what the task actually **is**.

Might be problem with the nature of the data, more specifically not enough data for practical learning considering the size of the (unmodified) feature space. We only have ~10x data points to the number of features.

Might want to try, say, a random forest or alternatively doing some tricks with sample generation. Might also be issues regarding colinearity, but that's a bit beyond me, to be quite hones.